In [1]:
!git clone https://github.com/rosinality/style-based-gan-pytorch.git

Cloning into 'style-based-gan-pytorch'...
remote: Enumerating objects: 189, done.
remote: Total 189 (delta 0), reused 0 (delta 0), pack-reused 189
Receiving objects: 100% (189/189), 34.80 MiB | 21.00 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [3]:
ls style-based-gan-pytorch/

checkpoint/  doc/         LICENSE   prepare_data.py  sample/
dataset.py   generate.py  model.py  README.md        train.py


In [8]:
!wget "https://www.dropbox.com/s/w8xqa8lgkzmhvl4/stylegan-1024px-new.model?dl=1" -O style-based-gan-pytorch/checkpoint/stylegan-1024px-new.model

--2020-05-11 02:19:01--  https://www.dropbox.com/s/w8xqa8lgkzmhvl4/stylegan-1024px-new.model?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6026:1::a27d:4601, 162.125.70.1
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6026:1::a27d:4601|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/w8xqa8lgkzmhvl4/stylegan-1024px-new.model [following]
--2020-05-11 02:19:01--  https://www.dropbox.com/s/dl/w8xqa8lgkzmhvl4/stylegan-1024px-new.model
Reusing existing connection to [www.dropbox.com]:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf7839019a686721c5b44c2ae50.dl.dropboxusercontent.com/cd/0/get/A3c6dBYcOFrMENgv3K66Yse3rzz1rvbhaf_hAN5MiXFA7_U44lvouvggfRYkoc2egwG58TOVR9JDEnabi27Qs1_rElTelbjSNyiT1K_7sMQsM2aYZkdV-PkhB72bP6cw0hc/file?dl=1# [following]
--2020-05-11 02:19:02--  https://ucf7839019a686721c5b44c2ae50.dl.dropboxusercontent.com/cd/0/get/A3c6dBYcOFrMENgv3K66Yse3rzz1rvbhaf_hAN5MiXFA7_U44lvouv

In [2]:
cd style-based-gan-pytorch/

/home/jruziev/gan/style-based-gan-pytorch


In [7]:
!python generate.py checkpoint/stylegan-1024px-new.model --size 1024 --n_row 2 --n_col 5

In [82]:
import torch
from torchvision import utils

from model import StyledGenerator

device = 'cuda:1'

generator = StyledGenerator(512).to(device)
generator.load_state_dict(torch.load("checkpoint/stylegan-1024px-new.model")['g_running'])
generator.eval();

In [84]:
@torch.no_grad()
def get_mean_style(generator, device):
    mean_style = None

    for i in range(10):
        style = generator.mean_style(torch.randn(1024, 512).to(device))

        if mean_style is None:
            mean_style = style

        else:
            mean_style += style

    mean_style /= 10
    return mean_style
mean_style = get_mean_style(generator, device)

In [49]:
point1 = torch.randn(512, device=device)

img1 = generator(point1.view(1,-1), 
                 step=8, 
                 mean_style=get_mean_style(generator, device),
                 alpha=1,
                 style_weight=0.5)
utils.save_image(
    img1, f'interpol1.png', nrow=1, normalize=True, range=(-1, 1)
)

In [55]:
point2 = torch.randn(512, device=device)

img2 = generator(point2.view(1,-1), 
                 step=8, 
                 mean_style=get_mean_style(generator, device),
                 alpha=1,
                 style_weight=0.5)
utils.save_image(
    img2, f'interpol2.png', nrow=1, normalize=True, range=(-1, 1)
)

In [96]:
with torch.no_grad():
    points = [point1.to(device) * w + point2.to(device) * (1 - w) for w in torch.linspace(0, 1, 12, device=device)]
    imgs = [generator(point.view(1,-1), 
                     step=8, 
                     mean_style=mean_style.to(device),
                     alpha=1,
                     style_weight=0.5) for point in points]
    utils.save_image(
        torch.cat(imgs, 0), f'interpol.png', nrow=4, normalize=True, range=(-1, 1)
    )

In [3]:
a228 = 228

In [8]:
a228

228